In [1]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd

# In Anaconda -> Environments -> Not Installed -> pandas-datareader -> Apply
from pandas_datareader import data as web # Reads stock data 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

import warnings
warnings.simplefilter("ignore")

# pip install yfinance in Qt Console or Powershell Prompt on Windows
import yfinance as yf

from plotly.subplots import make_subplots

# Install the Python Technical Analysis library
# pip install ta in Qt Console or Powershell prompt
from ta.trend import MACD
from ta.momentum import StochasticOscillator



## Download Stock data

In [2]:
amd_df = yf.download(tickers="AMD", period="1y", interval='1d')
amd_df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-02-19,89.750000,90.419998,88.690002,89.580002,89.580002,29548300
2021-02-22,88.150002,88.300003,85.209999,85.370003,85.370003,36930200
2021-02-23,83.400002,85.110001,79.360001,84.739998,84.739998,49720900
2021-02-24,84.330002,87.089996,82.809998,86.940002,86.940002,40821200
2021-02-25,86.169998,87.089996,81.919998,82.419998,82.419998,54625100
...,...,...,...,...,...,...
2022-02-14,115.510002,118.370003,113.459999,114.269997,114.269997,135146400
2022-02-15,117.190002,121.879997,114.360001,121.470001,121.470001,144139700
2022-02-16,119.050003,119.540001,114.220001,117.690002,117.690002,119109600


## Exponential Moving average
Allows us to see the big picture - puts more emphasis on recent price data (reduces the lag)


In [3]:
amd_df['MA20'] = amd_df['Adj Close'].ewm(span=20, adjust=False).mean()
amd_df['MA50'] = amd_df['Adj Close'].ewm(span=50, adjust=False).mean()


## Plot moving averages

In [7]:
fig = go.Figure()
candle = go.Candlestick(x=amd_df.index, open=amd_df['Open'], high=amd_df['High'], 
                        low=amd_df['Low'], close=amd_df['Close'], name='Candlestick')
ema20 = go.Scatter(x=amd_df.index, y=amd_df['MA20'], 
                    line=dict(color='green', width=1), name='EMA20')
ema50 = go.Scatter(x=amd_df.index, y=amd_df['MA50'], 
                    line=dict(color='orange', width=1), name='EMA50')
fig.add_trace(candle)
fig.add_trace(ema20)
fig.add_trace(ema50)
fig.update_layout(title='AMD')
fig.update_xaxes(rangebreaks=[dict(bounds=['sat', 'mon '])])
fig.show()


## Moving Average Convergence and Divergence

In [16]:
def plot_macd_stock(ticker, time_period, time_int):
    stock_df = yf.download(tickers=ticker, period=time_period, interval=time_int)
    candle = go.Candlestick(x=stock_df.index, open=stock_df['Open'], high=stock_df['High'], 
                        low=stock_df['Low'], close=stock_df['Close'], name='Candlestick')
    stock_df['MA12'] = stock_df['Close'].ewm(span=12, adjust=False).mean()
    stock_df['MA26'] = stock_df['Close'].ewm(span=26, adjust=False).mean()

    macd = MACD(close=stock_df['Close'], window_slow=26, window_fast=12, window_sign=9)

    sto_os = StochasticOscillator(high=stock_df['High'], close=stock_df['Close'], low=stock_df['Low'], window=14, smooth_window=3)

    fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.01)

    fig.add_trace(candle, row=1, col=1)

    fig.add_trace(go.Scatter(x=stock_df.index, y=macd.macd(), line=dict(color='blue', width=2)), row=2, col=1)

    fig.add_trace(go.Scatter(x=stock_df.index, y=macd.macd_signal(), line=dict(color='orange', width=2)), row=2, col=1)

    fig.add_trace(go.Bar(x=stock_df.index, y=macd.macd_diff()), row=2, col=1)

    fig.add_trace(go.Scatter(x=stock_df.index, y=sto_os.stoch(), line=dict(color='blue', width=2)), row=3, col=1)

    fig.add_trace(go.Scatter(x=stock_df.index, y=sto_os.stoch_signal(), line=dict(color='orange', width=2)), row=3, col=1)

    fig.add_hline(y=20, line_width=1, line_dash='dash', line_color='red', row=3, col=1)
    fig.add_hline(y=80, line_width=1, line_dash='dash', line_color='green', row=3, col=1)

    fig.add_trace(go.Bar(x=stock_df.index, y=stock_df['Volume']), row=4, col=1)

    fig.update_layout(title=ticker)
    fig.update_yaxes(title_text='Price', row=1, col=1)
    fig.update_yaxes(title_text='MACD', row=2, col=1)
    fig.update_yaxes(title_text='Stoch', row=3, col=1)
    fig.update_yaxes(title_text='Volume', row=4, col=1)

    fig.update_layout(height=900, width=1200, showlegend=False, xaxis_rangeslider_visible=False, 
                        xaxis_rangebreaks=[dict(bounds=['sat', 'mon']), dict(bounds=[16, 9.5], pattern='hour')])

    fig.show()




In [18]:
plot_macd_stock('PG', '2mo', '1h')

[*********************100%***********************]  1 of 1 completed


## What these charts are showing

- When we are looking at this chart we see some restance price-wise at 140. When we zoom into the specific data we can see that when the MACD crosses over the 140 value these will represent potential opportunities to buy or sell. 
- The Stochastic will also provide a signal of upcoming trend reversal. Values over 80 are considered to be over bought while values under 20 are considered to be undersold. This isn't a simple buy or sell plot as it's common for stocks to stay in a over-bought or over-sold position for very long periods. Best to use this chart as a backup to what you already have. It's very useful for when an investment is in a down or uptrend (security is bouncing up and down over an extended period of time but still reaching either higher highs or lower lows).  